### CÓDIGO TRANSFORMACIÓN FINAL DE LOS DATOS
#### TÍTULO: Los refugiados en los acuerdos de paz | MARIO MARTÍNEZ GARCÍA

In [1]:
#Cargamos las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re

df = pd.read_csv('pax_data_476_agreements_18-04-20.csv')

#Calculamos el año
df["Year"]=[df["Dat"][i][0:4] for i in range(len(df["Dat"]))]

#Seleccionamos las variables categóricas y numéricas juntándolas sobre una misma variable
categorical=['Con','Reg', 'Year','Loc1ISO','Loc2ISO', 'AgtId']
numerical=[ 'GCh', 'GRa', 'GRef', 'GRe', 'HrGen','EqGen', 'Med', 'Dev','SsrDdr']
variable=categorical+numerical


#Creamos nuestro conjunto transformado
data_trans=df[variable]
#Nombre de las variables que normalizaremos
norm_numerical=[ 'GCh_norm', 'GRa_norm', 'GRef_norm', 'GRe_norm', 'HrGen_norm','EqGen_norm', 'Med_norm', 'Dev_norm','SsrDdr_norm']
#Normalizamos las variables numéricas
x = data_trans[numerical]
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
#Introducimos las variables numéricas al conjunto
data_trans[norm_numerical] = pd.DataFrame(x_scaled)

#Elimanamos los paréntesis de los países
data_trans['Con']=[data_trans["Con"][i].replace('(','').replace(')','') for i in range(len(data_trans["Con"]))]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
#Cargamos el fichero con los países reales
con=pd.read_excel('Country.xlsx')

#Creamos un conjunto de listas vacías
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
x8=[]

#Definimos una función que nos muestra la presencia de elementos iguales
def todos_elementos_iguales(lista):
    longitud = len(lista)
    for i in range(longitud-1):
        if lista[i] != lista[i+1]:
                return False
    return True

#Realizamos un bucle que me compare los países reales con los países de data_trans y me guarde en
#las listas vacías los elementos descritos para cada país
for j in range(len(con["country"])):
    v=[re.search(con["country"][j], data_trans["Con"][i]) for i in range(len(data_trans["Con"]))]
    if todos_elementos_iguales(v)==False:
        for k in range(len(v)):
            if v[k]!=None: 
                x1.append(con["country"][j]) 
                x2.append(con["code"][j]) 
                x3.append(data_trans["Con"][k])                                                        
                x4.append(data_trans["AgtId"][k])  
                x5.append(data_trans["Year"][k])
                x6.append(data_trans["Reg"][k])
                x7.append(data_trans["GRef_norm"][k])
                x8.append(data_trans["HrGen_norm"][k])
                
#Creamos un diccionario para crear un data frame
peace_con= {"Country": x1 , 
                  "ISO code": x2 , 
                  "Con_AGR": x3, 
                  "AgtId": x4, 
                  "Year": x5,
                  "Reg": x6,
                  "GRef_norm": x7,
                  "HrGen_norm": x8,
            
                  }

#Creamos el dataframe
pcon=pd.DataFrame(peace_con)
pcon

,Country,ISO code,Con_AGR,AgtId,Year,Reg,GRef_norm,HrGen_norm
0,Afghanistan,AFG,Afghanistan,1739,2016,Europe and Eurasia,0.5,0.0
1,Afghanistan,AFG,Afghanistan,864,2012,Europe and Eurasia,0.0,1.0
2,Afghanistan,AFG,Afghanistan,848,2011,Europe and Eurasia,0.0,1.0
3,Afghanistan,AFG,Afghanistan,849,2011,Europe and Eurasia,0.0,1.0
4,Afghanistan,AFG,Afghanistan,709,2010,Europe and Eurasia,0.0,1.0
...,...,...,...,...,...,...,...,...
611,Zambia,ZMB,Angola/Burundi/Central African Republic/Democr...,349,2004,Africa (excl MENA),0.5,1.0
612,Zambia,ZMB,Angola/Burundi/Central African Republic/Democr...,789,2013,Africa (excl MENA),0.0,1.0
613,Zimbabwe,ZWE,Zimbabwe,1340,2013,Africa (excl MENA),1.0,1.0
614,Zimbabwe,ZWE,Zimbabwe,826,2008,Africa (excl MENA),0.5,1.0


In [3]:
#Eliminamos cross-regional
data_trans=data_trans.drop(data_trans[data_trans['Reg']=="Cross-regional"].index)
pcon=pcon.drop(pcon[pcon['Con_AGR']=='Pakistan/Taliban'].index)
#Transformamos a excel los datos transformados y el fichero de países
data_trans.to_excel('datatransform.xlsx')
pcon.to_excel('peace_con.xlsx')

In [4]:
#Comprobamos que disponíamos de todos los países en el nuevo fichero
len(data_trans["Con"].unique())==len(pcon['Con_AGR'].unique())

True

In [5]:
#De quedar algún país suelto sería indicado mediante este código
d=data_trans["Con"].unique()
p=pcon['Con_AGR'].unique()

for j in range(len(d)):
    v=[re.search(d[j], p[i]) for i in range(len(p))]
    if todos_elementos_iguales(v)==True:
        print(d[j])